In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Analyze collected data

## AMI NonVee

In [ ]:
#-------------------------
calculate_by_PN = True
#-------------------------

In [ ]:
# base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forAmanda\SN_882735506'
# base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\forMico2'
# base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\20230924_20230930\AllOPCOs'
# base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\20230813_20230819\AllOPCOs'
base_dir = r'C:\Users\s346557\Documents\LocalData\dovs_check\Mico2\AllOPCOs'

In [ ]:
base_dir_ami = os.path.join(base_dir, r'AMINonVee')
base_dir_ede = os.path.join(base_dir, r'EndEvents')
assert(os.path.exists(base_dir_ami))
assert(os.path.exists(base_dir_ede))

In [ ]:
if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl')):
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'rb') as handle:
        outg_rec_nb_to_files_dict = pickle.load(handle)
else:
    #-------------------------
    paths = Utilities.find_all_paths(
        base_dir=base_dir_ami, 
        glob_pattern=r'ami_nonvee_[0-9]*.csv', 
        regex_pattern=None
    )
    paths=natsorted(paths)
    #-------------------------
    outg_rec_nbs_in_files = dict()
    for path in paths:
        assert(path not in outg_rec_nbs_in_files.keys())
        df = GenAn.read_df_from_csv(path)
        outg_rec_nbs_in_files[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
    outg_rec_nb_to_files_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files)
    #-------------------------
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_dict.pkl'), 'wb') as handle:
        pickle.dump(outg_rec_nb_to_files_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-------------------------
all_outg_rec_nbs = list(outg_rec_nb_to_files_dict.keys())

In [ ]:
if os.path.exists(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl')):
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'rb') as handle:
        outg_rec_nb_to_files_ede_dict = pickle.load(handle)
else:
    #-------------------------
    paths_ede = Utilities.find_all_paths(
        base_dir=base_dir_ede, 
        glob_pattern=r'end_events_[0-9]*.csv', 
        regex_pattern=None
    )
    paths_ede=natsorted(paths_ede)
    #-------------------------
    outg_rec_nbs_in_files_ede = dict()
    for path in paths_ede:
        assert(path not in outg_rec_nbs_in_files_ede.keys())
        df = GenAn.read_df_from_csv(path)
        outg_rec_nbs_in_files_ede[path] = df['OUTG_REC_NB_GPD_FOR_SQL'].unique().tolist()
    outg_rec_nb_to_files_ede_dict = invert_file_to_outg_rec_nbs_dict(outg_rec_nbs_in_files_ede)
    #-------------------------
    with open(os.path.join(base_dir, 'outg_rec_nb_to_files_ede_dict.pkl'), 'wb') as handle:
        pickle.dump(outg_rec_nb_to_files_ede_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
#-------------------------
all_outg_rec_nbs_ede = list(outg_rec_nb_to_files_ede_dict.keys())

In [ ]:
#-------------------------
instvabc_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value=['INSTVA1', 'INSTVB1', 'INSTVC1'], 
            comparison_operator='isin'
        )
    ], 
    name='VOLT, INSTV(ABC)1', 
    join_single_slicers='and'
)
#-------------------------
volt_avg_slcr = DFSlicer(
    single_slicers = [
        dict(
            column='aep_derived_uom', 
            value='VOLT', 
            comparison_operator='=='
        ), 
        dict(
            column='aep_srvc_qlty_idntfr', 
            value='AVG', 
            comparison_operator='=='
        )
    ], 
    name='VOLT, AVG', 
    join_single_slicers='and'
)
#-------------------------
slicers=[instvabc_slcr, volt_avg_slcr]

In [ ]:
# start = time.time()
# outg_rec_nbs_to_remove = identify_outg_rec_nbs_to_remove(
#     paths=paths,
#     slicers=slicers, 
#     verbose=True
# )
# print(time.time()-start)

In [ ]:
# outg_rec_nb = '13619616'
# outg_rec_nb = '13618021'
outg_rec_nb = '13579980'

In [ ]:
outg_rec_nb_to_files_dict[outg_rec_nb]

In [ ]:
start=time.time()

ami_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_dict[outg_rec_nb])
#-------------------------
ami_df_i = ami_df[ami_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()

# Although I cannot yet call choose_best_slicer_and_perform_slicing and reduce_INSTV_ABC_1_vals_in_df, 
#   as the standard cleaning and conversions must be done first, I am able to cut down the size of
#   ami_df_i by joining the slicers with 'or' statements.
# Thus, ami_df_i will be reduced to only the subset of data which will be considered in 
#   choose_best_slicer_and_perform_slicing
# As mentioned, this will cut down the size of ami_df_i and will also save time and resources by not having
#   to run entire DF through cleaning and conversions procedures.
ami_df_i = DFSlicer.combine_slicers_and_perform_slicing(
    df=ami_df_i, 
    slicers=slicers, 
    join_slicers='or'
)


if ami_df_i.shape[0]==0:
    print('No usable data in ami_df_i')
    assert(0)

#-------------------------
ami_df_i = AMINonVee.perform_std_initiation_and_cleaning(ami_df_i)
#-----
# Should the following be added to AMINonVee.perform_std_initiation_and_cleaning?
ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ami_df_i, 
    time_col='starttimeperiod', 
    placement_col='starttimeperiod_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)
ami_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
    df=ami_df_i, 
    time_col='endtimeperiod', 
    placement_col='endtimeperiod_local', 
    run_quick=True, 
    n_strip=6, 
    inplace=False
)
idk = ami_df_i.copy()
#-------------------------
ami_df_i = choose_best_slicer_and_perform_slicing(
    df=ami_df_i, 
    slicers=slicers, 
    groupby_SN=True, 
    t_search_min_max=None, 
    time_col='starttimeperiod_local', 
    value_col=None, 
    SN_col='serialnumber', 
    return_sorted=True
)

ami_df_i = reduce_INSTV_ABC_1_vals_in_df(
    df=ami_df_i, 
    value_col='value', 
    aep_derived_uom_col='aep_derived_uom', 
    aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
    output_aep_srvc_qlty_idntfr = 'INSTV(ABC)1'
)


if ami_df_i.shape[0]==0:
    print('No usable data in ami_df_i')
    assert(0)
    
#-------------------------
# Each serial number should have a single value per time stamp
assert(ami_df_i.groupby(['serialnumber', 'starttimeperiod_local']).ngroups == ami_df_i.shape[0])

n_SNs  = ami_df_i['serialnumber'].nunique()
n_PNs  = ami_df_i['aep_premise_nb'].nunique()

print(time.time()-start)

In [ ]:
ami_df_i['serialnumber'].nunique()

In [ ]:
ami_df_i['aep_premise_nb'].nunique()

In [ ]:
ami_df_i[['serialnumber', 'aep_premise_nb']].drop_duplicates()['aep_premise_nb'].value_counts()

In [ ]:
ami_df_i.groupby(['serialnumber', 'starttimeperiod_local']).ngroups

In [ ]:
ami_df_i.shape

# To artificially add some periods during outage where single meter retores power for a period, use method below
# BEGIN

In [ ]:
# # ##########################################################################################
# # ami_df_i = ami_df_i.reset_index()
# # #-----
# # tmp = ami_df_i[
# #     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 02:30:00')) & 
# #     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:30:00')) &
# #     (ami_df_i['serialnumber']=='880694142')
# # ]
# # #-----
# # ami_df_i.loc[tmp.index,'value']=225
# # #-----
# # ami_df_i=ami_df_i.set_index('time_idx')
# # #-----


# ##########################################################################################
# ami_df_i = ami_df_i.reset_index()
# #-----
# tmp = ami_df_i[
#     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 02:30:00')) & 
#     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:00:00')) 
# ]
# #-----
# ami_df_i.loc[tmp.index,'value']=225
# #-----
# ami_df_i=ami_df_i.set_index('time_idx')
# #-----



# # ##########################################################################################
# # ami_df_i = ami_df_i.reset_index()
# # #-----
# # tmp = ami_df_i[
# #     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 02:30:00')) & 
# #     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:00:00')) 
# # ]
# # #-----
# # ami_df_i.loc[tmp.index,'value']=225
# # #-----
# # ami_df_i=ami_df_i.set_index('time_idx')
# # #-----

# # ami_df_i = ami_df_i.reset_index()
# # #-----
# # tmp = ami_df_i[
# #     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 02:30:00')) & 
# #     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:00:00')) &
# #     (ami_df_i['serialnumber']=='880694142')
# # ]
# # #-----
# # ami_df_i.loc[tmp.index,'value']=0
# # #-----
# # ami_df_i=ami_df_i.set_index('time_idx')
# # #-----



# # ##########################################################################################
# # ami_df_i = ami_df_i.reset_index()
# # #-----
# # tmp = ami_df_i[
# #     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 02:30:00')) & 
# #     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:00:00')) 
# # ]
# # #-----
# # ami_df_i.loc[tmp.index,'value']=225
# # #-----
# # ami_df_i=ami_df_i.set_index('time_idx')
# # #-----



# # ami_df_i = ami_df_i.reset_index()
# # #-----
# # tmp = ami_df_i[
# #     (ami_df_i['starttimeperiod_local']>=pd.to_datetime('2023-01-01 03:20:00')) & 
# #     (ami_df_i['starttimeperiod_local']<=pd.to_datetime('2023-01-01 03:40:00')) &
# #     (ami_df_i['serialnumber']=='880694142')
# # ]
# # #-----
# # ami_df_i.loc[tmp.index,'value']=225
# # #-----
# # ami_df_i=ami_df_i.set_index('time_idx')
# #-----

# END 

In [ ]:
if outg_rec_nb not in outg_rec_nb_to_files_ede_dict.keys():
    ede_df_i=None
else:
    ede_df = GenAn.read_df_from_csv_batch(outg_rec_nb_to_files_ede_dict[outg_rec_nb])
    ede_df_i = ede_df[ede_df['OUTG_REC_NB_GPD_FOR_SQL']==outg_rec_nb].copy()

    #-----
    ede_df_i = Utilities_dt.strip_tz_info_and_convert_to_dt(
        df=ede_df_i, 
        time_col='valuesinterval', 
        placement_col='valuesinterval_local', 
        run_quick=True, 
        n_strip=6, 
        inplace=False
    )
    ede_df_i = AMIEndEvents.reduce_end_event_reasons_in_df(df=ede_df_i)
    #-----
    ede_cols_to_keep = [
        'valuesinterval_local', 
        'reason', 
        'serialnumber', 
        'aep_premise_nb', 
        'enddeviceeventtypeid', 
        'event_type', 
        'OUTG_REC_NB_GPD_FOR_SQL', 
        'trsf_pole_nb_GPD_FOR_SQL',
    ]
    ede_df_i = ede_df_i[ede_cols_to_keep]

In [ ]:
# Build dovs_df_i
dovs_i = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_std_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=[outg_rec_nb], 
        field_to_split='outg_rec_nbs', 
    ), 
    build_consolidated=True
)
dovs_df_i = dovs_i.df.copy()
assert(dovs_df_i.shape[0]==1)

In [ ]:
dovs_i.df.columns.tolist()

In [ ]:
# Get the outage time from DOVS
dovs_outg_t_beg_end = dovs_df_i.iloc[0][['DT_OFF_TS_FULL', 'DT_ON_TS']].tolist()
assert(len(dovs_outg_t_beg_end)==2)
dovs_outg_t_beg, dovs_outg_t_end = dovs_outg_t_beg_end
print(f'dovs_outg_t_beg = {dovs_outg_t_beg}')
print(f'dovs_outg_t_end = {dovs_outg_t_end}')
#-------------------------
# Get the CI and CMI from DOVS
ci_cmi_dovs = dovs_df_i.iloc[0][['CI_NB', 'CMI_NB']].tolist()
assert(len(ci_cmi_dovs)==2)
ci_dovs, cmi_dovs = ci_cmi_dovs
print(f'ci_dovs  = {ci_dovs}')
print(f'cmi_dovs = {cmi_dovs}')
#-------------------------
# Get the number of premises from DOVS
n_PNs_dovs = len(set(dovs_df_i.iloc[0]['premise_nbs']))
#-------------------------
# Get the outage number from DOVS
outage_nb = dovs_df_i.iloc[0]['OUTAGE_NB']

In [ ]:
# ami_w_dovs_i = ami_df_i.merge(
#     dovs_df_i.drop(columns=['premise_nbs']), 
#     left_on='OUTG_REC_NB_GPD_FOR_SQL', 
#     right_index=True, 
#     how='left'
# )

In [ ]:
print(f"PNs with AMI data  : {ami_df_i['aep_premise_nb'].nunique()}")
print(f"PNs with end events: {ede_df_i['aep_premise_nb'].nunique()}")
print(f"PNs from DOVS: {n_PNs_dovs}")

In [ ]:
fig_num=0

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=fig_num)

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
i_subplot=0
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('120 hours'), 
    plot_time_beg_end=[dovs_outg_t_beg, dovs_outg_t_end], 
    data_label='', 
    title_args=dict(label='All', fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    include_outage_limits_text=True, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
ax.legend().set_visible(False)

fig_num+=1

In [ ]:
ami_df_i.shape

In [ ]:
# Estimate the outage times
pct_SNs_required_for_outage=0
est_outg_times = estimate_outage_times(
    df=ami_df_i, 
    t_search_min_max=None, 
    pct_SNs_required_for_outage=pct_SNs_required_for_outage, 
    relax_pct_SNs_required_if_no_outgs_found=True, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    verbose=True
)
print(est_outg_times)

In [ ]:
ci_NEW, cmi_NEW = calculate_ci_cmi_w_ami_w_ede_help(
    df=ami_df_i, 
    ede_df=None, 
    dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
    expand_outg_search_time=pd.Timedelta('1 hour'), 
    conservative_estimate=True, 
    return_CI_SNs=False, 
    use_est_outg_times=False, 
    pct_SNs_required_for_outage_est=0, 
#     expand_outg_est_search_time=pd.Timedelta('1 hour'), 
    expand_outg_est_search_time=pd.Timedelta('12 hours'), 
    use_only_overall_endpoints_of_est_outg_times=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
)

Expanding search window to include ending of last outage
SN = 880713230
	Org = 2023-09-04 12:16:00
	New = 2023-09-04 14:30:00


Expanding search window to include beginning of first outage
SN = 880713231
	Org = 2023-09-04 09:46:00
	New = 2023-09-03 19:30:00


Expanding search window to include ending of last outage
SN = 880713231
	Org = 2023-09-04 12:16:00
	New = 2023-09-04 14:30:00


Expanding search window to include beginning of first outage
SN = 880713232
	Org = 2023-09-04 09:46:00
	New = 2023-09-03 19:30:00


Expanding search window to include ending of last outage
SN = 880713232
	Org = 2023-09-04 12:16:00
	New = 2023-09-04 14:30:00


Expanding search window to include beginning of first outage
SN = 883459604
	Org = 2023-09-04 09:46:00
	New = 2023-09-03 19:30:00


Expanding search window to include ending of last outage
SN = 883459604
	Org = 2023-09-04 12:16:00
	New = 2023-09-04 14:30:00


In [ ]:
print('-----'*5+'\nDOVS\n'+'-----'*2)
print(f'CI  = {ci_dovs}')
print(f'CMI = {cmi_dovs}')

print('-----'*5+'\nAMI using new method\n'+'-----'*2)
print(f'CI  = {ci_NEW}')
print(f'CMI = {cmi_NEW}')

In [ ]:
cnsrvtv_out_t_beg = np.min([x['cnsrvtv_t_beg'] for x in est_outg_times]+[dovs_outg_t_beg])
cnsrvtv_out_t_end = np.max([x['cnsrvtv_t_end'] for x in est_outg_times]+[dovs_outg_t_end])

In [ ]:
# End events only
SN_col='serialnumber'
valuesinterval_col='valuesinterval_local' 
edetypeid_col='enddeviceeventtypeid'
#-------------------------
CMIs = []
ede_est_outg_times=dict()
for SN_ij in ede_df_i[SN_col].unique().tolist():
#     print(SN_ij)
    ede_df_ij = ede_df_i[ede_df_i[SN_col]==SN_ij]
    cmi_i, est_outg_times_i = estimate_cmi_using_ede_for_meter(
        ede_df_i=ede_df_ij, 
        broad_out_t_beg=cnsrvtv_out_t_beg, 
        broad_out_t_end=cnsrvtv_out_t_end, 
        expand_search_time=pd.Timedelta('1 hour'), 

        pd_ids=['3.26.0.47', '3.26.136.47'], 
        pu_ids=['3.26.0.216', '3.26.136.216'], 

        SN_col='serialnumber', 
        valuesinterval_col='valuesinterval_local', 
        edetypeid_col='enddeviceeventtypeid', 
        return_est_outage_times=True
    )
    CMIs.append(cmi_i)
    assert(SN_ij not in ede_est_outg_times.keys())
    ede_est_outg_times[SN_ij] = est_outg_times_i
tot_CMI = np.sum(CMIs)
tot_CMI = tot_CMI.total_seconds()/60
cmi_ede = tot_CMI
ci_ede = len([x for x in CMIs if x!=pd.Timedelta(0)])
print(f'CMI DOVS: {cmi_dovs}')
print(f'CMI EDE:  {tot_CMI}')
print(f'Delta:    {cmi_dovs-tot_CMI}')

In [ ]:
print('-----'*5+'\nDOVS\n'+'-----'*2)
print(f'CI  = {ci_dovs}')
print(f'CMI = {cmi_dovs}')

print('-----'*5+'\nAMI using new method\n'+'-----'*2)
print(f'CI  = {ci_NEW}')
print(f'CMI = {cmi_NEW}')

print('-----'*5+'\nEnd Events\n'+'-----'*2)
print(f'CI  = {ci_ede}')
print(f'CMI = {cmi_ede}')

In [ ]:
n_SNs_w_outg_times = 0
for SN_i, outg_times_i in ede_est_outg_times.items():
    print(f'SN = {SN_i}')
    print(f'n_outg_times = {len(outg_times_i)}')
    if len(outg_times_i)>0:
        n_SNs_w_outg_times+=1
    for time_i in outg_times_i:
        print(f'{time_i[0]} -- {time_i[1]}')
    print()

In [ ]:
ede_df_i.sort_values(by=['valuesinterval_local'])

In [ ]:
ede_df_i.sort_values(by=['valuesinterval_local']).iloc[70:90]

In [ ]:
# DRAWING ALL ede_est_outg_times!
fig, ax = Plot_General.default_subplots(fig_num=fig_num)

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('1 hour'), 
    plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    data_label='', 
    title_args=dict(label='All', fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    include_outage_limits_text=True, 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
ax.legend().set_visible(False)
add_est_outg_times_to_axis(
    ax, 
    list(itertools.chain.from_iterable(list(ede_est_outg_times.values()))), 
    include='both', 
    include_outage_limits_text=False
)

fig_num+=1

In [ ]:
dovs_outg_t_beg_end

In [ ]:
ede_est_outg_times

In [ ]:
ede_df_i.sort_values(by=['valuesinterval_local']).tail(10)

In [ ]:
all_outg_times_beg = []
all_outg_times_end = []
for SN_i, outg_times_i in ede_est_outg_times.items():
    for time_i in outg_times_i:
        all_outg_times_beg.append(time_i[0])
        all_outg_times_end.append(time_i[1])
min_outg_time = np.min(all_outg_times_beg)
max_outg_time = np.max(all_outg_times_end)
#-----
min_outg_time = np.min([min_outg_time, dovs_outg_t_beg])
max_outg_time = np.max([max_outg_time, dovs_outg_t_end])
#-----
min_outg_time -= pd.Timedelta('10 minutes')
max_outg_time += pd.Timedelta('10 minutes')
#-------------------------
fig,ax = Plot_General.default_subplots(  
    fig_num=fig_num, 
    unit_figsize_width=14,
    unit_figsize_height=24
)
ax.set_xlim(min_outg_time, max_outg_time);
ax.set_ylim(0, n_SNs_w_outg_times);
#-----
ax.axvline(dovs_outg_t_beg, color='red', linestyle='--')
ax.axvline(dovs_outg_t_end, color='green', linestyle='--')
#-------------------------
colors = Plot_General.get_standard_colors(n_colors=10, palette='colorblind')
#-------------------------
counter = 0
for SN_i, outg_times_i in ede_est_outg_times.items():
    if len(outg_times_i)>0:
        for time_i in outg_times_i:
            ax.hlines(y=counter, xmin=time_i[0], xmax=time_i[1], color=colors[counter % len(colors)], linestyle='-')
        counter += 1
        
fig_num+=1

In [ ]:
res_dict = calculate_ci_cmi_w_ami_w_ede_help(
    df=ami_df_i, 
    ede_df=ede_df_i, 
    dovs_outg_t_beg_end=dovs_outg_t_beg_end, 
    expand_outg_search_time=pd.Timedelta('1 hour'), 
    conservative_estimate=True, 
    est_ede_kwargs=None, 
    audit_selection_method='ede only', 
    return_CI_SNs=False, 
    use_est_outg_times=True, 
    pct_SNs_required_for_outage_est=0, 
#     expand_outg_est_search_time=pd.Timedelta('1 hour'), 
    expand_outg_est_search_time=pd.Timedelta('12 hours'), 
    use_only_overall_endpoints_of_est_outg_times=False, 
    t_int_beg_col='starttimeperiod_local', 
    t_int_end_col='endtimeperiod_local', 
    value_col='value', 
    SN_col='serialnumber', 
    return_all_best_ests=True, 
    return_all_best_ests_type='pd.DataFrame'
)
#-----
ci_ami       = res_dict['CI']
cmi_ami      = res_dict['CMI']
best_ests_df = res_dict['all_best_ests']

In [ ]:
print(f'ci_ami  = {ci_ami}')
print(f'cmi_ami = {cmi_ami}')

In [ ]:
best_ests_df

In [ ]:
best_ests_df['SN'].nunique()

In [ ]:
best_ests_df['PN'].nunique()

In [ ]:
if calculate_by_PN and best_ests_df.shape[0]>0:
    best_ests_df = combine_PNs_in_best_ests_df(
        best_ests_df, 
        likeness_thresh = pd.Timedelta('15 minutes'), 
        SN_col = 'SN', 
        PN_col = 'PN', 
        i_outg_col = 'i_outg'     
    )
    ci_ami  = best_ests_df['PN'].nunique()
    cmi_ami = (best_ests_df['winner_max']-best_ests_df['winner_min']).sum().total_seconds()/60

In [ ]:
print(f'ci_ami  = {ci_ami}')
print(f'cmi_ami = {cmi_ami}')

In [ ]:
if best_ests_df.shape[0]>0:
    best_ests_df_dovs_beg = alter_best_ests_df_using_dovs_outg_t_beg(
        best_ests_df=best_ests_df,
        dovs_df=dovs_df_i, 
        outg_rec_nb=outg_rec_nb
    )
    if calculate_by_PN:
        ci_ami_dovs_beg  = best_ests_df_dovs_beg['PN'].nunique()
    else:
        ci_ami_dovs_beg  = best_ests_df_dovs_beg['SN'].nunique()
    cmi_ami_dovs_beg = (best_ests_df_dovs_beg['winner_max']-best_ests_df_dovs_beg['winner_min']).sum().total_seconds()/60
else:
    best_ests_df_dovs_beg = best_ests_df.copy()
    ci_ami_dovs_beg = ci_ami
    cmi_ami_dovs_beg = cmi_ami

In [ ]:
print(f'ci_ami_dovs_beg  = {ci_ami_dovs_beg}')
print(f'cmi_ami_dovs_beg = {cmi_ami_dovs_beg}')

In [ ]:
best_ests_df

In [ ]:
# DRAWING ALL ESTIMATES
fig, ax = Plot_General.default_subplots(fig_num=fig_num)

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
i_subplot=0
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('1 hour'), 
    plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    data_label='', 
    title_args=dict(label='All', fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text=dict(
        out_t_beg_text='DOVS Beg.', 
        out_t_beg_ypos=(-0.1, 'ax_coord'), 
        out_t_beg_va='top', 
        out_t_beg_ha='center', 
        out_t_beg_color='red', 
        #-----
        out_t_end_text='DOVS End ', 
        out_t_end_ypos=(-0.1, 'ax_coord'), 
        out_t_end_va='top', 
        out_t_end_ha='center', 
        out_t_end_color='green', 
    ), 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
ax.legend().set_visible(False)
add_all_best_ests_to_axis(
    ax, 
    best_ests_df, 
    line_kwargs_by_est_key=dict(
        conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
        zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
    ), 
    keys_to_include=['winner', 'conservative', 'zero_times']
)

fig_num+=1

In [ ]:
eps_min=5
min_samples=2
ests_to_include_in_clustering = ['winner_min', 'winner_max']
ests_to_include_in_output = ['winner_min', 'winner_max', 'conservative_min', 'conservative_max', 'zero_times_min', 'zero_times_max']

In [ ]:
if best_ests_df.shape[0]>0:
    means_df, best_ests_df_w_db_lbl = get_mean_times_w_dbscan(
        best_ests_df, 
        eps_min=5, 
        min_samples=2, 
        ests_to_include_in_clustering=['winner_min', 'winner_max'],
        ests_to_include_in_output=[
            'winner_min', 'winner_max', 
            'conservative_min', 'conservative_max', 
            'zero_times_min', 'zero_times_max'
        ], 
        return_labelled_best_ests_df=True
    )
else:
    means_df, best_ests_df_w_db_lbl = None, None

In [ ]:
if best_ests_df_dovs_beg.shape[0]>0:
    means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = get_mean_times_w_dbscan(
        best_ests_df_dovs_beg, 
        eps_min=5, 
        min_samples=2, 
        ests_to_include_in_clustering=['winner_min', 'winner_max'],
        ests_to_include_in_output=[
            'winner_min', 'winner_max', 
            'conservative_min', 'conservative_max', 
            'zero_times_min', 'zero_times_max'
        ], 
        return_labelled_best_ests_df=True
    )
else:
    means_df_dovs_beg, best_ests_df_dovs_beg_w_db_lbl = None, None

In [ ]:
summary_df_i = build_detailed_summary_df(
    means_df=means_df, 
    best_ests_df_w_db_lbl=best_ests_df_w_db_lbl,
    CI_tot=ci_ami, 
    CMI_tot=cmi_ami, 
    n_PNs_ami=n_PNs, 
    outg_rec_nb=outg_rec_nb, 
    dovs_df_i=dovs_df_i,
    db_label_col='db_label', 
    winner_min_col='winner_min', 
    winner_max_col='winner_max', 
    PN_col='PN' if calculate_by_PN else 'SN', 
    i_outg_col='i_outg'
)
summary_df_i

In [ ]:
# DRAWING ONLY MEANS
fig, ax = Plot_General.default_subplots(fig_num=fig_num)

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
i_subplot=0
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('1 hour'), 
    plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    data_label='', 
    title_args=dict(label='All', fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text=dict(
        out_t_beg_text='DOVS Beg.', 
        out_t_beg_ypos=(-0.1, 'ax_coord'), 
        out_t_beg_va='top', 
        out_t_beg_ha='center', 
        out_t_beg_color='red', 
        #-----
        out_t_end_text='DOVS End ', 
        out_t_end_ypos=(-0.1, 'ax_coord'), 
        out_t_end_va='top', 
        out_t_end_ha='center', 
        out_t_end_color='green', 
    ), 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
ax.legend().set_visible(False)
add_all_best_ests_to_axis(
    ax, 
    means_df, 
    line_kwargs_by_est_key=dict(
        conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
        zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
    ), 
    keys_to_include=['winner', 'conservative', 'zero_times']
)

#-------------------------
patch_dovs_beg = Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=1.0, linestyle='-', 
    label='DOVS Beg.'
)
patch_dovs_end = Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=1.0, linestyle='-', 
    label='DOVS End'
)
#-----
patch_ui_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='Beg. Uncertainty Interval'
)
patch_ui_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='End Uncertainty Interval'
)
#-----
patch_best_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. Beg.'
)
patch_best_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. End'
)
#-------------------------
leg_1 = ax.legend(
    title=None, 
    handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end], 
    bbox_to_anchor=(1, 0.975), 
    loc='upper left'
)

fig_num+=1

In [ ]:
fig, axs = Plot_General.default_subplots(fig_num=fig_num, n_x=1, n_y=means_df.shape[0])
if means_df.shape[0]==1:
    axs = [axs]

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
for i_row in range(means_df.shape[0]):
    db_label = means_df.iloc[i_row].name
    ami_df_i_subset = ami_df_i[ami_df_i['aep_premise_nb'].isin(
        best_ests_df_w_db_lbl[best_ests_df_w_db_lbl['db_label']==db_label]['PN'].tolist()
    )]
    
    fig, axs[i_row] = AMINonVee.plot_usage_around_outage(
        fig=fig, 
        ax=axs[i_row], 
        data=ami_df_i_subset, 
        x='starttimeperiod_local', 
        y='value', 
        hue='serialnumber', 
        out_t_beg=dovs_outg_t_beg, 
        out_t_end=dovs_outg_t_end, 
        expand_time=pd.Timedelta('30 minutes'), 
        plot_time_beg_end=[means_df.iloc[i_row]['winner_min'], means_df.iloc[i_row]['winner_max']], 
        data_label='', 
        title_args=None, 
        ax_args=None, 
        xlabel_args=None, 
        ylabel_args=None, 
        df_mean=None, 
        df_mean_col=None, 
        mean_args=None, 
        draw_outage_limits=True, 
        draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text=dict(
            out_t_beg_text='DOVS Beg.', 
            out_t_beg_ypos=(-0.1, 'ax_coord'), 
            out_t_beg_rot=0, 
            out_t_beg_va='top', 
            out_t_beg_ha='center', 
            out_t_beg_color='red', 
            #-----
            out_t_end_text='DOVS End', 
            out_t_end_ypos=(-0.1, 'ax_coord'), 
            out_t_end_rot=0, 
            out_t_end_va='top', 
            out_t_end_ha='center', 
            out_t_end_color='green', 
        ),
        draw_without_hue_also=False, 
        seg_line_freq=None, 
        palette=palette
    )
    axs[i_row].legend().set_visible(False)
    add_all_best_ests_to_axis(
        axs[i_row], 
        means_df.iloc[[i_row]], 
        line_kwargs_by_est_key=dict(
            conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.5), 
            zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.5) 
        ), 
        keys_to_include=['winner', 'conservative', 'zero_times']
    )

fig_num+=1

In [ ]:
# Sanity (not good check in this instance, but in general, if multiple on/off times, would expect the DF
# to be larger than the number of unique values)
# CHECK THIS IN APPROPRIATE ISNTANCE
print(best_ests_df_w_db_lbl.shape[0])
print(best_ests_df_w_db_lbl['PN'].nunique())

In [ ]:
means_df

In [ ]:
# FINALS?!

In [ ]:
# pdf = PdfPages(r'C:\Users\s346557\Documents\LocalData\dovs_check\Results\Test.pdf')

In [ ]:
if means_df is not None:
    cnsrvtv_out_t_beg = means_df.min().min()
    cnsrvtv_out_t_end = means_df.max().max()
else:
    cnsrvtv_out_t_beg = dovs_outg_t_beg
    cnsrvtv_out_t_end = dovs_outg_t_end

fig, ax = Plot_General.default_subplots()

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
i_subplot=0
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('1 hour'), 
    plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    data_label='', 
    title_args=dict(label='All', fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
#     include_outage_limits_text=True, 
    include_outage_limits_text=dict(
        out_t_beg_text='DOVS Beg.', 
        out_t_beg_ypos=(-0.1, 'ax_coord'), 
        out_t_beg_va='top', 
        out_t_beg_ha='center', 
        out_t_beg_color='red', 
        #-----
        out_t_end_text='DOVS End', 
        out_t_end_ypos=(-0.1, 'ax_coord'), 
        out_t_end_va='top', 
        out_t_end_ha='center', 
        out_t_end_color='green', 
    ), 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
ax.legend().set_visible(False)
add_all_best_ests_to_axis(
    ax, 
    means_df, 
    line_kwargs_by_est_key=dict(
        conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
        zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
    ), 
    keys_to_include=['winner', 'conservative', 'zero_times']
)

#-------------------------
patch_dovs_beg = Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=1.0, linestyle='-', 
    label='DOVS Beg.'
)
patch_dovs_end = Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=1.0, linestyle='-', 
    label='DOVS End'
)
#-----
patch_ui_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='Beg. Uncertainty Interval'
)
patch_ui_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='End Uncertainty Interval'
)
#-----
patch_best_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. Beg.'
)
patch_best_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. End'
)
#-------------------------
leg_1 = ax.legend(
    title=None, 
    handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end], 
    bbox_to_anchor=(1, 0.975), 
    loc='upper left'
)

# pdf.savefig(fig, bbox_inches='tight')

In [ ]:
if means_df is not None:
    cnsrvtv_out_t_beg = means_df.min().min()
    cnsrvtv_out_t_end = means_df.max().max()
else:
    cnsrvtv_out_t_beg = dovs_outg_t_beg
    cnsrvtv_out_t_end = dovs_outg_t_end

# Instead of using get_full_part_not_outage_subset_dfs, simply grab the PNs which suffered
#   outages from best_ests_df
if best_ests_df.shape[0]>0:
    outg_SNs = best_ests_df['PN'].unique().tolist()
else:
    outg_SNs = []
#-----
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]
#-------------------------
print(f'nSNs Total               = {ami_df_i["serialnumber"].nunique()}')
print(f'ratio: {ami_df_i_out["serialnumber"].nunique()/ami_df_i["serialnumber"].nunique()}')

In [ ]:
if means_df is not None:
    cnsrvtv_out_t_beg = means_df.min().min()
    cnsrvtv_out_t_end = means_df.max().max()
else:
    cnsrvtv_out_t_beg = dovs_outg_t_beg
    cnsrvtv_out_t_end = dovs_outg_t_end

fig, axs = Plot_General.default_subplots(
    n_x=1,
    n_y=3,
    fig_num=fig_num,
    sharex=False,
    sharey=False,
    unit_figsize_width=14,
    unit_figsize_height=6, 
    return_flattened_axes=True,
    row_major=True
)
Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

palette = Plot_General.get_standard_colors_dict(
    keys=ami_df_i['serialnumber'].unique().tolist(), 
    palette='colorblind'
)

#-------------------------
i_subplot=0
fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=axs[i_subplot], 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('1 hour'), 
    plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
    data_label='', 
    title_args=dict(label=f"All (#SNs = {ami_df_i['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
    ax_args=None, 
    xlabel_args=None, 
    ylabel_args=None, 
    df_mean=None, 
    df_mean_col=None, 
    mean_args=None, 
    draw_outage_limits=True, 
    draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text=dict(
        out_t_beg_text='DOVS Beg.', 
        out_t_beg_ypos=(0.12, 'ax_coord'), 
        out_t_beg_va='bottom', 
        out_t_beg_ha='center', 
        out_t_beg_color='red', 
        #-----
        out_t_end_text='DOVS End', 
        out_t_end_ypos=(0.12, 'ax_coord'), 
        out_t_end_va='bottom', 
        out_t_end_ha='center', 
        out_t_end_color='green', 
    ), 
    draw_without_hue_also=False, 
    seg_line_freq=None, 
    palette=palette
)
axs[i_subplot].legend().set_visible(False)
Plot_General.set_general_plotting_args(
    ax=axs[i_subplot], 
    tick_args =[
        dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
        dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
    ], 
    xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
    ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
)


#-------------------------
i_subplot=1
if ami_df_i_out.shape[0]>0:
    fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
        fig=fig, 
        ax=axs[i_subplot], 
        data=ami_df_i_out, 
        x='starttimeperiod_local', 
        y='value', 
        hue='serialnumber', 
        out_t_beg=dovs_outg_t_beg, 
        out_t_end=dovs_outg_t_end, 
        expand_time=pd.Timedelta('1 hour'), 
        plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
        data_label='', 
        title_args=dict(label=f"Out (#SNs = {ami_df_i_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
        ax_args=None, 
        xlabel_args=None, 
        ylabel_args=None, 
        df_mean=None, 
        df_mean_col=None, 
        mean_args=None, 
        draw_outage_limits=True, 
        draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text=dict(
            out_t_beg_text='DOVS Beg.', 
            out_t_beg_ypos=(0.12, 'ax_coord'), 
            out_t_beg_va='bottom', 
            out_t_beg_ha='center', 
            out_t_beg_color='red', 
            #-----
            out_t_end_text='DOVS End', 
            out_t_end_ypos=(0.12, 'ax_coord'), 
            out_t_end_va='bottom', 
            out_t_end_ha='center', 
            out_t_end_color='green', 
        ), 
        draw_without_hue_also=False, 
        seg_line_freq=None, 
        palette=palette
    )
    axs[i_subplot].legend().set_visible(False)
    add_all_best_ests_to_axis(
        axs[i_subplot], 
        means_df, 
        line_kwargs_by_est_key=dict(
            conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
            zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
        ), 
        keys_to_include=['winner', 'conservative', 'zero_times']
    )
    Plot_General.set_general_plotting_args(
        ax=axs[i_subplot], 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
        ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
    )
else:
    axs[i_subplot].set_title(
        label=f'Out', 
        fontdict=dict(fontsize=24)
    )

#-------------------------
i_subplot=2
if ami_df_i_not_out.shape[0]>0:
    fig, axs[i_subplot] = AMINonVee.plot_usage_around_outage(
        fig=fig, 
        ax=axs[i_subplot], 
        data=ami_df_i_not_out, 
        x='starttimeperiod_local', 
        y='value', 
        hue='serialnumber', 
        out_t_beg=dovs_outg_t_beg, 
        out_t_end=dovs_outg_t_end, 
        expand_time=pd.Timedelta('1 hour'), 
        plot_time_beg_end=[cnsrvtv_out_t_beg, cnsrvtv_out_t_end], 
        data_label='', 
        title_args=dict(label=f"Not Out (#SNs = {ami_df_i_not_out['serialnumber'].nunique()})", fontdict=dict(fontsize=24)), 
        ax_args=None, 
        xlabel_args=None, 
        ylabel_args=None, 
        df_mean=None, 
        df_mean_col=None, 
        mean_args=None, 
        draw_outage_limits=True, 
        draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
        include_outage_limits_text=dict(
            out_t_beg_text='DOVS Beg.', 
            out_t_beg_ypos=(0.12, 'ax_coord'), 
            out_t_beg_va='bottom', 
            out_t_beg_ha='center', 
            out_t_beg_color='red', 
            #-----
            out_t_end_text='DOVS End', 
            out_t_end_ypos=(0.12, 'ax_coord'), 
            out_t_end_va='bottom', 
            out_t_end_ha='center', 
            out_t_end_color='green', 
        ), 
        draw_without_hue_also=False, 
        seg_line_freq=None, 
        palette=palette
    )
    axs[i_subplot].legend().set_visible(False)
    Plot_General.set_general_plotting_args(
        ax=axs[i_subplot], 
        tick_args =[
            dict(axis='x', labelrotation=0, labelsize=14.0, direction='out'), 
            dict(axis='y', labelrotation=0, labelsize=14.0, direction='out')
        ], 
        xlabel_args=dict(xlabel=axs[i_subplot].get_xlabel(), fontsize=16), 
        ylabel_args=dict(ylabel=axs[i_subplot].get_ylabel(), fontsize=16)
    )
else:
    axs[i_subplot].set_title(label='Not Out', fontdict=dict(fontsize=24))
        

#--------------------------------------------------
# Add legend to first plot
patch_dovs_beg = Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=5.0, linestyle='-', 
    label='DOVS Beg.'
)
patch_dovs_end = Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=5.0, linestyle='-', 
    label='DOVS End'
)
#-----
patch_ui_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='Beg. Uncertainty Interval'
)
patch_ui_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=5.0, linestyle=':', 
    label='End Uncertainty Interval'
)
#-----
patch_best_beg =  Line2D(
    [0], [0], color='red', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. Beg.'
)
patch_best_end =  Line2D(
    [0], [0], color='green', 
    alpha=1.0, linewidth=1.0, linestyle='--', 
    label='Best Est. End'
)
#-------------------------
handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end]
#-------------------------
leg_1 = axs[0].legend(
    title=None, 
    handles=handles, 
    bbox_to_anchor=(1, 1.025), 
    loc='upper left', 
    fontsize=15
)        
        
#--------------------------------------------------
ci_info_fontsize = 20
left_text_x=0.95
right_text_x = 1.05
shift_text_down = 0

fig.text(left_text_x, 0.745-shift_text_down, f'OUTG_REC_NB: {outg_rec_nb}', fontsize=ci_info_fontsize+4)
fig.text(left_text_x, 0.715-shift_text_down, f"OUTAGE_NB:     {outage_nb}", fontsize=ci_info_fontsize+4)

fig.text(left_text_x, 0.675-shift_text_down, f"#PNs from DOVS = {n_PNs_dovs}", fontsize=ci_info_fontsize)

fig.text(left_text_x, 0.640-shift_text_down, "----- Found in AMI -----", fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.615-shift_text_down, f"#PNs = {n_PNs}", fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.590-shift_text_down, f"#SNs = {n_SNs}", fontsize=ci_info_fontsize)

fig.text(left_text_x, 0.525-shift_text_down, '-----'*5+'\nDOVS\n'+'-----'*5, fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.500-shift_text_down, f'CI    = {ci_dovs}', fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.475-shift_text_down, f'CMI = {np.round(cmi_dovs, decimals=2)}', fontsize=ci_info_fontsize)

fig.text(left_text_x, 0.400-shift_text_down, '-----'*5+'\nUsing AMI\n'+'-----'*5, fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.375-shift_text_down, f'CI    = {ci_ami}', fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.350-shift_text_down, f'CMI = {np.round(cmi_ami, decimals=2)}', fontsize=ci_info_fontsize)
#-----
fig.text(
    left_text_x, 0.325-shift_text_down, 
    f'$\Delta$CI    = {ci_dovs-ci_ami} ({np.round(100*(ci_dovs-ci_ami)/ci_dovs, decimals=2)}%)', 
    fontsize=ci_info_fontsize
)
fig.text(
    left_text_x, 0.300-shift_text_down, 
    f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami)/cmi_dovs, decimals=2)}%)', 
    fontsize=ci_info_fontsize
)

fig.text(left_text_x, 0.225-shift_text_down, '-----'*5+'\nAMI w/ DOVS t_beg\n'+'-----'*5, fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.200-shift_text_down, f'CI    = {ci_ami_dovs_beg}', fontsize=ci_info_fontsize)
fig.text(left_text_x, 0.175-shift_text_down, f'CMI = {np.round(cmi_ami_dovs_beg, decimals=2)}', fontsize=ci_info_fontsize)
#-----
fig.text(
    left_text_x, 0.150-shift_text_down, 
    f'$\Delta$CI    = {ci_dovs-ci_ami_dovs_beg} ({np.round(100*(ci_dovs-ci_ami_dovs_beg)/ci_dovs, decimals=2)}%)', 
    fontsize=ci_info_fontsize
)
fig.text(
    left_text_x, 0.125-shift_text_down, 
    f'$\Delta$CMI = {np.round(cmi_dovs-cmi_ami_dovs_beg, decimals=2)} ({np.round(100*(cmi_dovs-cmi_ami_dovs_beg)/cmi_dovs, decimals=2)}%)', 
    fontsize=ci_info_fontsize
)

fig_num += 1

In [ ]:
all_detailed_summary_dfs=[]
ci_cmi_summary_df = pd.DataFrame(columns=[
    'outg_rec_nb', 
    'ci_dovs',  'ci_ami', 'ci_ami_dovs_beg', 
    'cmi_dovs', 'cmi_ami', 'cmi_ami_dovs_beg'
])


In [ ]:
if means_df is not None:
    means_df = means_df.sort_values(by=['winner_min', 'winner_max'])
    fig, axs = Plot_General.default_subplots(
        n_x=2, 
        n_y=means_df.shape[0], 
        fig_num=fig_num
    )
    if means_df.shape[0]==1:
        axs = [axs]
    Plot_General.adjust_subplots_args(fig, dict(hspace=0.30))

    palette = Plot_General.get_standard_colors_dict(
        keys=ami_df_i['serialnumber'].unique().tolist(), 
        palette='colorblind'
    )

    #-------------------------
    for i_row in range(means_df.shape[0]):
        db_label = means_df.iloc[i_row].name
        ami_df_i_subset = ami_df_i[ami_df_i['aep_premise_nb'].isin(
            best_ests_df_w_db_lbl[best_ests_df_w_db_lbl['db_label']==db_label]['PN'].tolist()
        )]
        n_SNs_i = ami_df_i_subset['serialnumber'].nunique()
        #****************************************
        fig, axs[i_row][0] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_row][0], 
            data=ami_df_i_subset, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('15 minutes'), 
            plot_time_beg_end=[means_df.iloc[i_row]['conservative_min'], means_df.iloc[i_row]['zero_times_min']], 
            data_label='', 
            title_args=None, 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ),
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_row][0].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_row][0], 
            means_df.iloc[[i_row]], 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=['winner', 'conservative', 'zero_times'], 
            expand_ax_to_accommodate=False
        )
        axs[i_row][0].text(0.85, 0.9, f'#SNs = {n_SNs_i}', ha='center', va='center', transform=axs[i_row][0].transAxes, fontsize='xx-large')
        Plot_General.set_general_plotting_args(
            ax=axs[i_row][0], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize='large', direction='out'), 
                dict(axis='y', labelrotation=0, labelsize='large', direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_row][0].get_xlabel(), fontsize='xx-large'), 
            ylabel_args=dict(ylabel=axs[i_row][0].get_ylabel(), fontsize='xx-large')
        )
        #****************************************
        fig, axs[i_row][1] = AMINonVee.plot_usage_around_outage(
            fig=fig, 
            ax=axs[i_row][1], 
            data=ami_df_i_subset, 
            x='starttimeperiod_local', 
            y='value', 
            hue='serialnumber', 
            out_t_beg=dovs_outg_t_beg, 
            out_t_end=dovs_outg_t_end, 
            expand_time=pd.Timedelta('15 minutes'), 
            plot_time_beg_end=[means_df.iloc[i_row]['zero_times_max'], means_df.iloc[i_row]['conservative_max']], 
            data_label='', 
            title_args=None, 
            ax_args=None, 
            xlabel_args=None, 
            ylabel_args=None, 
            df_mean=None, 
            df_mean_col=None, 
            mean_args=None, 
            draw_outage_limits=True, 
            draw_outage_limits_kwargs=dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
            include_outage_limits_text=dict(
                out_t_beg_text='DOVS Beg.', 
                out_t_beg_ypos=(0.12, 'ax_coord'), 
                out_t_beg_va='bottom', 
                out_t_beg_ha='center', 
                out_t_beg_color='red', 
                #-----
                out_t_end_text='DOVS End', 
                out_t_end_ypos=(0.12, 'ax_coord'), 
                out_t_end_va='bottom', 
                out_t_end_ha='center', 
                out_t_end_color='green', 
            ),
            draw_without_hue_also=False, 
            seg_line_freq=None, 
            palette=palette
        )
        axs[i_row][1].legend().set_visible(False)
        add_all_best_ests_to_axis(
            axs[i_row][1], 
            means_df.iloc[[i_row]], 
            line_kwargs_by_est_key=dict(
                conservative=dict(alpha=0.25, linewidth=5.0, ymax=0.6), 
                zero_times=dict(alpha=0.25, linewidth=5.0, ymax=0.4) 
            ), 
            keys_to_include=['winner', 'conservative', 'zero_times'], 
            expand_ax_to_accommodate=False
        )
        axs[i_row][1].text(0.15, 0.9, f'#SNs = {n_SNs_i}', ha='center', va='center', transform=axs[i_row][1].transAxes, fontsize='xx-large')
        Plot_General.set_general_plotting_args(
            ax=axs[i_row][1], 
            tick_args =[
                dict(axis='x', labelrotation=0, labelsize='large', direction='out'), 
                dict(axis='y', labelrotation=0, labelsize='large', direction='out')
            ], 
            xlabel_args=dict(xlabel=axs[i_row][1].get_xlabel(), fontsize='xx-large'), 
            ylabel_args=dict(ylabel=axs[i_row][1].get_ylabel(), fontsize='xx-large')
        )
        
    #--------------------------------------------------
    # Add legend to first row
    patch_dovs_beg = Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS Beg.'
    )
    patch_dovs_end = Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle='-', 
        label='DOVS End'
    )
    #-----
    patch_ui_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='Beg. Uncertainty Interval'
    )
    patch_ui_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=5.0, linestyle=':', 
        label='End Uncertainty Interval'
    )
    #-----
    patch_best_beg =  Line2D(
        [0], [0], color='red', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. Beg.'
    )
    patch_best_end =  Line2D(
        [0], [0], color='green', 
        alpha=1.0, linewidth=1.0, linestyle='--', 
        label='Best Est. End'
    )
    #-------------------------
    leg_1 = axs[0][1].legend(
        title=None, 
        handles=[patch_dovs_beg, patch_dovs_end, patch_ui_beg, patch_ui_end, patch_best_beg, patch_best_end], 
        bbox_to_anchor=(1, 1.025), 
        loc='upper left', 
        fontsize=15
    )    
        
    #-------------------------
    fig.suptitle(f"OUTG_REC_NB: {outg_rec_nb}", y=0.95, fontsize='xx-large')
    #-------------------------
    # pdf.savefig(fig, bbox_inches='tight')
    
    fig_num+=1

In [ ]:
# pdf.close()

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = dict(
        ci_ami   = ci_ami_dovs_beg, 
        cmi_ami  = cmi_ami_dovs_beg, 
        means_df = means_df_dovs_beg, 
        name = 'AMI w/ DOVS t_beg'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times']
)
fig_num += 1

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
    means_df             = means_df_dovs_beg, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami_dovs_beg, 
    cmi_ami              = cmi_ami_dovs_beg, 
    name                 = 'AMI w/ DOVS t_beg', 
    results_2_dict       = dict(
        ci_ami   = ci_ami, 
        cmi_ami  = cmi_ami, 
        means_df = means_df, 
        name = 'AMI'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner']
)
fig_num += 1

In [ ]:
# assert(0)

# BEG: DEV calculating how many meters have power at any time

In [ ]:
def build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
):
    r"""
    ami_df_i
        - Only needed for finding n_PNs_tot
    """
    #-------------------------
    df = best_ests_df[[PN_col, i_outg_col, t_min_col, t_max_col]].copy()
    #-------------------------
    all_times_sorted = natsorted(set(df[t_min_col].tolist()+df[t_max_col].tolist()))
    all_times_sorted = [all_times_sorted[0]-pd.Timedelta('1T')] + all_times_sorted
    #-----
    # For each unique time, determine how many PNs without power
    n_PNs_tot = ami_df_i[PN_col_ami_df].nunique()
    time_and_n_PNs_w_power=dict()
    for time_i in all_times_sorted:
        df_i = df[
            (df[t_min_col]<=time_i) & 
            (df[t_max_col]>time_i)
        ].copy()
        # Should find at most one outage for a given PN!
        assert(df_i.shape[0]==df_i[PN_col].nunique())
        n_PNs_out_i = df_i.shape[0]
        assert(time_i not in time_and_n_PNs_w_power.keys())
        if return_pct:
            time_and_n_PNs_w_power[time_i] = 100*(n_PNs_tot-n_PNs_out_i)/n_PNs_tot
        else:
            time_and_n_PNs_w_power[time_i] = n_PNs_tot-n_PNs_out_i
    #-------------------------
    n_PNs_w_power_srs = pd.Series(data=time_and_n_PNs_w_power)
    #-------------------------
    return n_PNs_w_power_srs

def simplify_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    freq='1T'
):
    r"""
    If there are a bunch of entries close together, this will keep only the one with the largest value.
    Intention: To be used when including data point information in text on plot for n_PNs_w_power_srs
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    if n_PNs_w_power_srs.name:
        name = n_PNs_w_power_srs.name
    else:
        name = 'pct_w_power'
    #-------------------------
    n_PNs_w_power_srs_simp = n_PNs_w_power_srs.to_frame(name=name).reset_index().groupby(
        pd.Grouper(freq=freq, key='index')
    ).apply(
        lambda x: x.loc[x[name].idxmax()] if x.shape[0]>0 else None
    )
    n_PNs_w_power_srs_simp = n_PNs_w_power_srs_simp.dropna()
    n_PNs_w_power_srs_simp=n_PNs_w_power_srs_simp.set_index('index').squeeze()
    #-------------------------
    return n_PNs_w_power_srs_simp


def get_periods_above_threshold(
    n_PNs_w_power_srs, 
    threshold, 
    return_indices=False
):
    r"""
    Given n_PNs_w_power_srs, returns the beginning and ending of periods during which the number
    of premises with power is above threshold
    
    n_PNs_w_power_srs:
        A series with index equal to timestamps and values equal to the percent of premises with power
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    n_PNs_w_power_srs = n_PNs_w_power_srs.copy()
    if n_PNs_w_power_srs.name:
        pct_w_power_col = n_PNs_w_power_srs.name
    else:
        pct_w_power_col = Utilities.generate_random_string()
        n_PNs_w_power_srs.name = pct_w_power_col
    #-------------------------
    above_thresh_col = 'above_thresh'
    above_thresh_df = (n_PNs_w_power_srs>threshold).astype(int).to_frame()
    above_thresh_df=above_thresh_df.rename(columns={pct_w_power_col:above_thresh_col})
    #-----
    tmp_diff_col = Utilities.generate_random_string()
    above_thresh_df[tmp_diff_col] = above_thresh_df[above_thresh_col].diff()
    #-------------------------
    # The .diff() operation always leaves the first element as a NaN
    # However, if the first element pct_w_power>threshold it represents the beginning
    #   of a black and the diff should be +1
    if above_thresh_df.iloc[0][above_thresh_col]==1:
        above_thresh_df.loc[above_thresh_df.index[0], tmp_diff_col] = 1
    #-------------------------
    # Continuous blocks of True (i.e., blocks with pct_w_power > threshold) begin with diff = +1 and
    #   end at the element preceding diff = -1
    block_beg_idxs = above_thresh_df.reset_index().index[above_thresh_df[tmp_diff_col]==1].tolist()
    #-----
    block_end_idxs = above_thresh_df.reset_index().index[above_thresh_df[tmp_diff_col]==-1].tolist()
    block_end_idxs = [x-1 for x in block_end_idxs]
    # If power is ongoing at the end of the data, the procedure above will miss the last end idx
    #   If single point above threshold at end of data ==> tmp_diff_col = +1
    #   If multiple points above threshold at the end of the data, then tmp_diff_col for the last value will be 0
    #     In this case, there does not exist a tmp_diff_col=-1 to signal the end of the block, so must add by hand
    if above_thresh_df.iloc[-1][above_thresh_col]==1:
        block_end_idxs.append(above_thresh_df.shape[0]-1)
    #--------------------------------------------------
    # periods of length one should have idx in both block_beg_idxs and block_end_idxs
    len_1_pd_idxs = natsorted(set(block_beg_idxs).intersection(set(block_end_idxs)))
    #-------------------------
    # Remove the len_1 idxs so the remaineders can be matched
    # NOTE: The following procedure relies on block_beg(end)_idxs being sorting
    block_beg_idxs = natsorted(set(block_beg_idxs).difference(len_1_pd_idxs))
    block_end_idxs = natsorted(set(block_end_idxs).difference(len_1_pd_idxs))
    assert(len(block_beg_idxs)==len(block_end_idxs))
    block_begend_idxs = list(zip(block_beg_idxs, block_end_idxs))
    #-------------------------
    # Include the length 1 blocks!
    block_begend_idxs.extend([(x,x) for x in len_1_pd_idxs])
    #-------------------------
    # Sort
    block_begend_idxs = natsorted(block_begend_idxs, key=lambda x: x[0])
    #-------------------------
    # Sanity check!
    for i in range(len(block_begend_idxs)):
        assert(len(block_begend_idxs[i])==2)
        assert(block_begend_idxs[i][1]>=block_begend_idxs[i][0])
        if i>0:
            assert(block_begend_idxs[i][0]>block_begend_idxs[i-1][1])
    #-------------------------
    # Convert indices to actual values
    block_begend = [(above_thresh_df.index[x[0]], above_thresh_df.index[x[1]]) for x in block_begend_idxs]
    #-------------------------
    if return_indices:
        return block_begend_idxs
    else:
        return block_begend
    
def get_first_last_above_threshold(
    n_PNs_w_power_srs, 
    threshold
):
    r"""
    Get the first and last time (after initial power loss) that power was regained.
    For simple outages, these should be approximately equal
    """
    #-------------------------
    pds_above_thresh = get_periods_above_threshold(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        threshold=threshold, 
        return_indices=False
    )
    #----------------------------------------------------------------------------------------------------
    # len(pds_above_thresh)==2:
    #     For simple outages, where all initially have power, power is lost suddenly for all, and power is regained 
    #       suddenly for all, there should be two periods above threshold (len(pds_above_thresh)==2)
    #
    # len(pds_above_thresh)==1:
    #     This can arise from a few different situations (#2 is the only usable of the 3):
    #       1: - 'Power' is never lost (i.e., the percent of premises with power never dips below threshold)
    #          -  The single period spans the data
    #          -  The period data is of no use
    #       2: - 'Power' is already out at the beginning of the data 
    #          -  The beginning of the single period may be used to signify when power is restored
    #       3: - 'Power' is on at the beginning of the data, goes out, but is never restored
    #          -  The period data is of no use
    #
    # len(pds_above_thresh)==0:
    #     'Power' is out for the entirety of the data
    #
    # len(pds_above_thresh)>2:
    #     More complicated structure with sub-outages
    #
    #-------------------------
    # How we interpret the data depends on whether there is power at the beginng and end of the data
    power_at_beg = n_PNs_w_power_srs.iloc[0]>threshold
    power_at_end = n_PNs_w_power_srs.iloc[-1]>threshold
    #-------------------------
    if len(pds_above_thresh)==0:
        assert(not power_at_beg and not power_at_end)
        frst_above=None
        last_above=None
    #-------------------------
    elif len(pds_above_thresh)==1:
        assert(power_at_beg or power_at_end)
        last_above=None
        # If power_at_beg and power_at_end, then 'power' is never lost ==> situation #1 (described above)
        # If power_at_beg==False and power_at_end==True  ==> situation #2 (described above)
        # If power_at_beg==True  and power_at_end==False ==> situation #3 (described above)
        if power_at_beg==False and power_at_end==True:
            frst_above = pds_above_thresh[0][0]
        else:
            frst_above = None
    #-------------------------
    else:
        assert(power_at_beg or power_at_end)
        #-----
        # First time above threshold:
        #   If first data point is above threshold, then return beginning of the second period (at index=1)
        #     - i.e., if there is 'power' at the beginning of the data, return the first time power was restored
        #         after it was lost (where 'power' is defined as a minimum threshold of premises having power)
        #   If first data point is not above threshold, the power was initially out, so return beginning of first 
        #     period (at index=0)
        if power_at_beg:
            frst_above = pds_above_thresh[1][0]
        else:
            frst_above = pds_above_thresh[0][0]
        #-----
        # Last time above threshold:
        # If last data point is above threshold, then return beginning of last period (at index=-1)
        #     - i.e., if there is 'power' at the end of the data, return the beginning of the last period
        # If the last data point is not above threshold, the 'power' is out at the end of the data, so
        #   in this case return None
        if power_at_end:
            last_above = pds_above_thresh[-1][0]
        else:
            last_above = None
    #-------------------------
    return frst_above, last_above
    
def get_n_PNs_w_power_srs_time_to_print(
    n_PNs_w_power_srs, 
    threshold
):
    r"""
    In figure, entries with n_PNs_w_power>threshold are typically circled and have both their pct and time
    printed next to the data point.
    However, if there are multiple in a row, this can be messy.
    In such cases, we only want to print the time for those at the beginning and end of each grouping.
    This function identifies which entries should have their times printed
    """
    #-------------------------
    assert(isinstance(n_PNs_w_power_srs, pd.Series))
    #-------------------------
    pds_above_thresh = get_periods_above_threshold(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        threshold=threshold, 
        return_indices=True
    )
    # Get unique idxs from pds_above_thresh
    print_times = natsorted(set(itertools.chain.from_iterable(pds_above_thresh)))
    #-------------------------
    # Always print the first and last times!
    if 0 not in print_times:
        print_times = [0]+print_times
    if n_PNs_w_power_srs.shape[0]-1 not in print_times:
        print_times = print_times+[n_PNs_w_power_srs.shape[0]-1]
    #-------------------------
    return_print_times_srs = n_PNs_w_power_srs.iloc[print_times]
    #-------------------------
    return return_print_times_srs


def build_n_PNs_w_power_srs_and_plot(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    simp_freq='1T', 
    threshold=None, 
    fig_num=0, 
    fig_ax=None, 
    title=None, 
    adjust_texts=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb', 
    **plot_kwargs
):
    r"""
    ami_df_i
        - Only needed for finding n_PNs_tot
    """
    #-------------------------
    n_PNs_w_power_srs = build_n_PNs_w_power_srs(
        best_ests_df  = best_ests_df, 
        ami_df_i      = ami_df_i, 
        return_pct    = return_pct, 
        PN_col        = PN_col, 
        t_min_col     = t_min_col, 
        t_max_col     = t_max_col, 
        i_outg_col    = i_outg_col, 
        PN_col_ami_df = PN_col_ami_df
    )
    #-----
    if not n_PNs_w_power_srs.name:
        n_PNs_w_power_srs.name = 'pct_w_power'
    #-------------------------    
    fig,ax = DOVSAudit.static_plot_n_PNs_w_power_srs(
        n_PNs_w_power_srs=n_PNs_w_power_srs, 
        simp_freq=simp_freq, 
        threshold=threshold, 
        fig_num=fig_num, 
        title=title, 
        adjust_texts=adjust_texts, 
        fig_ax=fig_ax, 
        **plot_kwargs
    )
    #-------------------------
    return n_PNs_w_power_srs, fig, ax

In [ ]:
threshold=95

In [ ]:
threshold=95

n_PNs_w_power_srs = build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)
#-----
if not n_PNs_w_power_srs.name:
    n_PNs_w_power_srs.name = 'pct_w_power'

In [ ]:
fig,ax = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=0
)

In [ ]:
n_PNs_w_power_srs, fig, ax = build_n_PNs_w_power_srs_and_plot(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=0, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)

In [ ]:
n_PNs_w_power_srs, fig, ax = build_n_PNs_w_power_srs_and_plot(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=0, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb', 
    threshold_color='magenta'
)

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = dict(
        ci_ami   = ci_ami_dovs_beg, 
        cmi_ami  = cmi_ami_dovs_beg, 
        means_df = means_df_dovs_beg, 
        name = 'AMI w/ DOVS t_beg'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915  
)

n_PNs_w_power_srs, fig, axs[3] = build_n_PNs_w_power_srs_and_plot(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb', 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

In [ ]:
fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = cnsrvtv_out_t_beg, 
    cnsrvtv_out_t_end    = cnsrvtv_out_t_end, 
    means_df             = means_df, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = dict(
        ci_ami   = ci_ami_dovs_beg, 
        cmi_ami  = cmi_ami_dovs_beg, 
        means_df = means_df_dovs_beg, 
        name = 'AMI w/ DOVS t_beg'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

# END: DEV

In [ ]:
def remove_any_dovs_overlaps_from_best_ests(
    best_ests_df, 
    outg_rec_nb, 
    dovs_df, 
    get_ptntl_ovrlp_dovs_kwargs=None, 
    assert_no_overlaps=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_dovs='PREMISE_NB', 
    t_min_col_dovs='DT_OFF_TS_FULL', 
    t_max_col_dovs='DT_ON_TS', 
    outg_rec_nb_col_dovs='OUTG_REC_NB', 
    overlap_outg_col='overlap_DOVS', 
    overlap_times_col='overlap_times', 
    keep_col='keep'
):
    r"""
    If dovs_df is not supplied by user, it will built.
        Default behavior is to query using the premise number and time restrictions from best_ests_df.
        However, one could override this behavior by supplying different arguments for get_ptntl_ovrlp_dovs_kwargs.
        
    If dovs_df is supplied by the user:
        - it must include outg_rec_nb
        - it must contain a 'PREMISE_NB' column
        
    get_ptntl_ovrlp_dovs_kwargs:
        A dictionary with key/value pairs suitable for input into get_potential_overlapping_dovs.
        The keys and default values are:
            - PNs: 
                -- premise numbers to query
                -- default = best_ests_df[PN_col].unique().tolist()
            - outg_t_beg: 
                -- beginning time to look for overlaps
                -- default = best_ests_df[t_min_col].min()
            - outg_t_end: 
                -- ending time to look for overlaps
                -- default = best_ests_df[t_max_col].max()
            - dovs_sql_fcn: 
                -- SQL functin to use for query 
                -- default = DOVSOutages_SQL.build_sql_std_outage
            - addtnl_dovs_sql_kwargs: 
                -- Any additional kwargs to input into dovs_sql_fcn when running query
                -- default = None
                -- NOTE: premise_nbs, field_to_split, dt_on_ts, and dt_off_ts_full are handled by 
                         get_potential_overlapping_dovs and should therefore NOT be included in 
                         addtnl_dovs_sql_kwargs
    
    NOTE: Using build_sql_std_outage by default, so typical restrictions on outages types etc.
    """
    #--------------------------------------------------
    return_df = DOVSAudit.identify_dovs_overlaps_from_best_ests(
        best_ests_df                = best_ests_df, 
        outg_rec_nb                 = outg_rec_nb, 
        dovs_df                     = dovs_df, 
        get_ptntl_ovrlp_dovs_kwargs = get_ptntl_ovrlp_dovs_kwargs, 
        assert_no_overlaps          = assert_no_overlaps, 
        PN_col                      = PN_col, 
        t_min_col                   = t_min_col, 
        t_max_col                   = t_max_col, 
        PN_col_dovs                 = PN_col_dovs, 
        t_min_col_dovs              = t_min_col_dovs, 
        t_max_col_dovs              = t_max_col_dovs, 
        outg_rec_nb_col_dovs        = outg_rec_nb_col_dovs, 
        overlap_outg_col            = overlap_outg_col, 
        overlap_times_col           = overlap_times_col, 
        keep_col                    = keep_col
    )
    #--------------------------------------------------
    # Locate overlap_outg_col, overlap_times_col, and keep_col in return_df
    #-----
    found_overlap_outg_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(overlap_outg_col)
    )
    assert(len(found_overlap_outg_col)==1)
    overlap_outg_col = found_overlap_outg_col[0]
    #-----
    found_overlap_times_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(overlap_times_col)
    )
    assert(len(found_overlap_times_col)==1)
    overlap_times_col = found_overlap_times_col[0]
    #-----
    found_keep_col = Utilities.find_in_list_with_regex(
        lst=return_df.columns.tolist(), 
        regex_pattern=r'{}.*'.format(keep_col)
    )
    assert(len(found_keep_col)==1)
    keep_col = found_keep_col[0]
    #--------------------------------------------------
    # Keep only those with keep_col values equal to True
    return_df = return_df[return_df[keep_col]==True].copy()
    #-----
    # No longer need overlap_outg_col, overlap_times_col, or keep_col, as they are trivially equal to [],[],True for all
    return_df = return_df.drop(columns=[overlap_outg_col, overlap_times_col, keep_col])
    #--------------------------------------------------
    # Re-build i_outg_col to account for any sub-outages removed
    return_df = DOVSAudit.set_i_outg_in_best_ests_df(
        best_ests_df = return_df, 
        groupby_cols = [PN_col], 
        sort_cols    = [t_min_col, t_max_col], 
        i_outg_col   = i_outg_col
    )
    #--------------------------------------------------
    return return_df

# Procedure
1. Look for any potential overlapping DOVS events
    - This simply looks at the premise numbers contained in ami_df_i and finds if any other DOVS
      events overlap with outg_rec_nb
    - TODO: This should only depend on the DOVS outg_rec_nb, and should have nothing to do with best_ests_df,
      which it currently does
    - Need to input min on CI/CMI
2. If there are any overlaps (whether in PNs which actually suffer from outage or those that do not), document
   in some new file
3. If there are overlaps in the PNs which actually suffer, then the analysis should stop
4. If there are no overlaps, or the overlaps only exist in the not-out subsampe, then run remove_any_dovs_overlaps_from_best_ests methodology

# TODO: CI/CMI>0!!!!!!!!!!!!

# TODO: NEED to check/protect against case where somehow a DOVS outage encompasses others?

In [ ]:
# dovs_sql_fcn=DOVSOutages_SQL.build_sql_std_outage
dovs_sql_fcn=DOVSOutages_SQL.build_sql_outage
PN_col='PN'
t_min_col='winner_min'
t_max_col='winner_max'
i_outg_col='i_outg'
PN_col_dovs='PREMISE_NB'
t_min_col_dovs='DT_OFF_TS_FULL'
t_max_col_dovs='DT_ON_TS'
PN_col_ami='aep_premise_nb'

In [ ]:
# I'll need potential overlapping DOVS df using (dovs_outg_t_beg, dovs_outg_t_end) and also
#   using (best_ests_df[t_min_col].min(), best_ests_df[t_max_col].max())
# Since these typically have significant overlap, grab potential overlapping DOVS events for 
#   all, and then subset as needed
ptntl_ovrlp_dovs_df = DOVSAudit.get_potential_overlapping_dovs(
    PNs=ami_df_i['aep_premise_nb'].unique().tolist(), 
    outg_t_beg=np.min([dovs_outg_t_beg, best_ests_df['winner_min'].min()]), 
    outg_t_end=np.max([dovs_outg_t_end, best_ests_df['winner_max'].max()]), 
    dovs_sql_fcn=dovs_sql_fcn, 
)
#-----
ptntl_ovrlp_dovs_df_out_times = ptntl_ovrlp_dovs_df[
    (ptntl_ovrlp_dovs_df['DT_OFF_TS_FULL'] <= dovs_outg_t_end) & 
    (ptntl_ovrlp_dovs_df['DT_ON_TS']       >= dovs_outg_t_beg)
]
ptntl_ovrlp_dovs_df_est_times = ptntl_ovrlp_dovs_df[
    (ptntl_ovrlp_dovs_df['DT_OFF_TS_FULL'] <= best_ests_df['winner_max'].max()) & 
    (ptntl_ovrlp_dovs_df['DT_ON_TS']       >= best_ests_df['winner_min'].min())
]       

In [ ]:
# First, find if any other DOVS outages overlap with the current one
#-----
# NOTE: Supply best_ests_df to get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN below so 'lost_power'
#       column is output
overlap_outgs_for_PNs_df = DOVSAudit.get_outgs_in_dovs_df_overlapping_outg_rec_nb_i_by_PN(
    outg_rec_nb_i=outg_rec_nb, 
    dovs_df=ptntl_ovrlp_dovs_df_out_times, 
    best_ests_df=best_ests_df, 
    outg_rec_nb_col='OUTG_REC_NB', 
    PN_col='PREMISE_NB', 
    t_min_col='DT_OFF_TS_FULL', 
    t_max_col='DT_ON_TS', 
    PN_col_best_ests='PN'
)
#-------------------------
# If any PN has one or more overlapping DOVS events, output info to file
n_PNs_w_overlap = (overlap_outgs_for_PNs_df['n_overlap']>0).sum()
if n_PNs_w_overlap>0:
    print(f'Need to output to new file\n\tn_PNs_w_overlap={n_PNs_w_overlap}')

In [ ]:
#-------------------------    
# If any PN which lost power has one or more overlapping DOVS events, stop analysis
n_out_PNs_w_overlap = overlap_outgs_for_PNs_df[overlap_outgs_for_PNs_df['lost_power']==True]['overlap_outg_rec_nbs'].apply(lambda x: len(x)>0).sum()
if n_out_PNs_w_overlap>0:
    print('STOP analysis')

In [ ]:
overlap_outgs_for_PNs_df['overlap_outg_rec_nbs'].apply(lambda x: len(x)>0).sum()

In [ ]:
assert(0)

In [ ]:
best_ests_df_OG = best_ests_df.copy()

In [ ]:
# Procedure above ensure that the current outage, as defined by DOVS, does not overlap with any other DOVS outages.
# Now, we are safe to check whether any of our estimates for the current outage overlap with any other DOVS outages!
# ==> Should be safe to set assert_no_overlaps=True in remove_any_dovs_overlaps_from_best_ests below
# NOTE: We don't care if there are overlap in the subset of data which does not actually suffer from the outages
best_ests_df_w_keep_info = DOVSAudit.identify_dovs_overlaps_from_best_ests(
    best_ests_df=best_ests_df, 
    outg_rec_nb=outg_rec_nb, 
    dovs_df=ptntl_ovrlp_dovs_df_est_times
)
best_ests_df = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==True].drop(
    columns=['overlap_DOVS', 'overlap_times', 'keep']
)

In [ ]:
threshold=95

n_PNs_w_power_srs = build_n_PNs_w_power_srs(
    best_ests_df, 
    ami_df_i, 
    return_pct=True, 
    PN_col='PN', 
    t_min_col='winner_min', 
    t_max_col='winner_max', 
    i_outg_col='i_outg', 
    PN_col_ami_df='aep_premise_nb'
)
#-----
if not n_PNs_w_power_srs.name:
    n_PNs_w_power_srs.name = 'pct_w_power'

In [ ]:
#-------------------------
# In ami_df_i, mark any entries which were essentially removed via the DOVSAudit.identify_dovs_overlaps_from_best_ests
#   and removal procedure above
ami_df_i = DOVSAudit.set_removed_due_to_overlap_in_ami_df_i(
    ami_df_i=ami_df_i, 
    best_ests_df=best_ests_df_w_keep_info.copy(), 
    PN_col='aep_premise_nb', 
    time_idfr='starttimeperiod_local', 
    PN_col_be='PN', 
    keep_col_be='keep', 
    overlap_times_col_be='overlap_times', 
    removed_due_to_overlap_col='removed_due_to_overlap'
)

In [ ]:
ami_df_i['removed_due_to_overlap'].sum()

In [ ]:
outg_SNs = best_ests_df['PN'].unique().tolist()
ami_df_i_out      = ami_df_i[ami_df_i['aep_premise_nb'].isin(outg_SNs)]
ami_df_i_not_out  = ami_df_i[~ami_df_i['aep_premise_nb'].isin(outg_SNs)]  

In [ ]:
means_df_fnl_2, best_ests_df_fnl_2_w_db_lbl = get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)

fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df_w_keep_info['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df_w_keep_info['conservative_max'].max(), 
    means_df             = means_df_fnl_2, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = dict(
        ci_ami   = ci_ami_dovs_beg, 
        cmi_ami  = cmi_ami_dovs_beg, 
        means_df = means_df_dovs_beg, 
        name = 'AMI w/ DOVS t_beg'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col='removed_due_to_overlap', 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

fig_num += 1

In [ ]:
# Why didn't DOVSAudit.expand_removed_srs operate properly in DOVSAudit.set_removed_due_to_overlap_in_ami_df_i?
# Why are there entries in Not Out which appear to suffer from outage?

In [ ]:
# #-------------------------
# # In ami_df_i, mark any entries which were essentially removed via the DOVSAudit.identify_dovs_overlaps_from_best_ests
# #   and removal procedure above
# ami_df_i = DOVSAudit.set_removed_due_to_overlap_in_ami_df_i(
#     ami_df_i=ami_df_i, 
#     best_ests_df=best_ests_df_w_keep_info.copy(), 
#     PN_col='aep_premise_nb', 
#     time_idfr='starttimeperiod_local', 
#     PN_col_be='PN', 
#     keep_col_be='keep', 
#     overlap_times_col_be='overlap_times', 
#     removed_due_to_overlap_col='removed_due_to_overlap'
# )

In [ ]:
removed_srs = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].copy()
removed_srs = removed_srs.groupby('PN')['overlap_times'].sum()

In [ ]:
best_ests_df_w_keep_info[best_ests_df_w_keep_info['PN']=='040587963']

In [ ]:
tmp_dovs = DOVSOutages(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True,
    build_sql_function=DOVSOutages_SQL.build_sql_outage, 
    build_sql_function_kwargs=dict(
        outg_rec_nbs=[outg_rec_nb, '13551140'], 
        include_DOVS_PREMISE_DIM=True
    ), 
    build_consolidated=True
)
tmp_dovs_df = tmp_dovs.df.copy()

In [ ]:
tmp_dovs_df

In [ ]:
tmp_PNs = list(set(tmp_dovs_df.iloc[0]['premise_nbs']).intersection(set(tmp_dovs_df.iloc[1]['premise_nbs'])))
tmp_PNs

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=fig_num)
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[ami_df_i['aep_premise_nb'].isin(tmp_PNs)], 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('12 hours'), 
    plot_time_beg_end=[dovs_outg_t_beg, dovs_outg_t_end], 
)

In [ ]:
removed_srs

In [ ]:
removed_srs_2 = DOVSAudit.expand_removed_srs(
    removed_srs  = removed_srs.copy(), 
    ami_df_i     = ami_df_i, 
    best_ests_df = best_ests_df_w_keep_info, 
)

In [ ]:
removed_srs_2

In [ ]:
winner_minmax_by_PN = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==True].groupby('PN').agg(
    {'winner_min':'min', 'winner_max':'max'}
)
#-----
ami_minmax_times_by_PN = ami_df_i.groupby('aep_premise_nb').agg(
    {'starttimeperiod_local':['min', 'max']}
)

In [ ]:
ami_df_i_not_out.groupby('aep_premise_nb').apply(lambda x: (x['value']==0).any())

In [ ]:
import AMINonVee 
reload(AMINonVee)
from AMINonVee import AMINonVee

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=fig_num)
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i[ami_df_i['aep_premise_nb'].isin(tmp_PNs)], 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('12 hours'), 
    plot_time_beg_end=[dovs_outg_t_beg, dovs_outg_t_end],
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text = dict(
        out_t_beg_text  = 'DOVS 1 Beg.', 
        out_t_beg_ypos  = (0.12, 'ax_coord'), 
        out_t_beg_va    = 'bottom', 
        out_t_beg_ha    = 'center', 
        out_t_beg_color = 'red', 
        #-----
        out_t_end_text  = 'DOVS 1 End', 
        out_t_end_ypos  = (0.12, 'ax_coord'), 
        out_t_end_va    = 'bottom', 
        out_t_end_ha    = 'center', 
        out_t_end_color = 'green', 
    ), 
    only_connect_continuous=True, 
)

In [ ]:
%run ./check_DOVS_METHODS.ipynb

In [ ]:
means_df_fnl_2, best_ests_df_fnl_2_w_db_lbl = get_mean_times_w_dbscan(
    best_ests_df, 
    eps_min=5, 
    min_samples=2, 
    ests_to_include_in_clustering=['winner_min', 'winner_max'],
    ests_to_include_in_output=[
        'winner_min', 'winner_max', 
        'conservative_min', 'conservative_max', 
        'zero_times_min', 'zero_times_max'
    ], 
    return_labelled_best_ests_df=True
)

fig, axs = plot_all_out_not_NEW(
    fig_num              = fig_num, 
    ami_df_i             = ami_df_i, 
    ami_df_i_out         = ami_df_i_out, 
    ami_df_i_not_out     = ami_df_i_not_out, 
    dovs_outg_t_beg      = dovs_outg_t_beg, 
    dovs_outg_t_end      = dovs_outg_t_end, 
    cnsrvtv_out_t_beg    = best_ests_df_w_keep_info['conservative_min'].min(), 
    cnsrvtv_out_t_end    = best_ests_df_w_keep_info['conservative_max'].max(), 
    means_df             = means_df_fnl_2, 
    outg_rec_nb          = outg_rec_nb, 
    outage_nb            = outage_nb, 
    n_PNs_dovs           = n_PNs_dovs, 
    ci_dovs              = ci_dovs, 
    cmi_dovs             = cmi_dovs, 
    ci_ami               = ci_ami, 
    cmi_ami              = cmi_ami, 
    name                 = 'AMI', 
    results_2_dict       = dict(
        ci_ami   = ci_ami_dovs_beg, 
        cmi_ami  = cmi_ami_dovs_beg, 
        means_df = means_df_dovs_beg, 
        name = 'AMI w/ DOVS t_beg'
    ), 
    expand_time          = pd.Timedelta('1 hour'), 
    mean_keys_to_include = ['winner', 'conservative', 'zero_times'], 
    removed_due_to_overlap_col='removed_due_to_overlap', 
    default_subplots_args=dict(n_x=2, n_y=2, row_major=True), 
    leg_i_plot=1, 
    leg_kwargs=dict(ncols=1, fontsize=15, bbox_to_anchor=(1, 1.2)), 
    ci_info_fontsize=16, 
    left_text_x=0.915
    
)

fig,axs[3] = DOVSAudit.static_plot_n_PNs_w_power_srs(
    n_PNs_w_power_srs, 
    simp_freq='1T', 
    threshold=threshold, 
    fig_num=fig_num, 
    fig_ax=(fig, axs[3]), 
    threshold_color='magenta'
)

axs[0].text(
    pd.to_datetime('2023-08-16 08:44:00'), 
    axs[0].transLimits.inverted().transform((0,0.12))[1], 
    '13551139 Beg.', 
    rotation=90, verticalalignment='bottom', horizontalalignment='center', color='red'
)
axs[0].text(
    pd.to_datetime('2023-08-16 14:39:00'), 
    axs[0].transLimits.inverted().transform((0,0.12))[1], 
    '13551139 End', 
    rotation=90, verticalalignment='bottom', horizontalalignment='center', color='green'
)



axs[0].axvline(pd.to_datetime('2023-08-16 15:44:00'), color='red', alpha=1.0, linewidth=5.0, ymax=0.1)
axs[0].text(
    pd.to_datetime('2023-08-16 15:44:00'), 
    axs[0].transLimits.inverted().transform((0,0.12))[1], 
    '13551140 Beg.', 
    rotation=90, verticalalignment='bottom', horizontalalignment='center', color='red'
)
axs[0].axvline(pd.to_datetime('2023-08-16 20:48:00'), color='green', alpha=1.0, linewidth=5.0, ymax=0.1)
axs[0].text(
    pd.to_datetime('2023-08-16 20:48:00'), 
    axs[0].transLimits.inverted().transform((0,0.12))[1], 
    '13551140 End', 
    rotation=90, verticalalignment='bottom', horizontalalignment='center', color='green'
)

fig_num += 1

In [ ]:
tmp_dovs_df

In [ ]:
assert(0)

# END DEV

In [ ]:
best_ests_df_w_keep_info_OG = best_ests_df_w_keep_info.copy()

In [ ]:
we = ami_df_i_not_out.groupby('aep_premise_nb').apply(lambda x: (x['value']==0).any())

In [ ]:
natsorted(ami_df_i[ami_df_i['aep_premise_nb'].isin(we[we==False].index.tolist())]['serialnumber'].unique().tolist())

In [ ]:
best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False]

In [ ]:
winner_min    = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['winner_min']
winner_max    = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['winner_max']
overlap_times = best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False].iloc[0]['overlap_times']

In [ ]:
def remove_overlap_from_interval_w_split(
    interval, 
    overlap, 
    split_interval=False
):
    r"""
    Not really used anywhere.
    This is how remove_overlap_from_interval was originally written, but I reduced the functionality to
      simplify things.
    So, this exists in case I desire the functionality later.
    
    Given an input interval and possible overlap interval, remove overlap from the interval.
    Both interval and overlap should be lists/tuples of length 2 whose elements represent the beginning and
      ending of the interval.
    
    By default, this will not split the input interval.
        If the overlap does split the interval, the portion of interval occurring prior to the overlap will be returned.
        The intent of this function is to remove any DOVS events overlapping with the algorithm outage times.
        Therefore, in such a split situation, we would not want anything after the first overlap.
    --------------------------------------------------
    There are basically 4 distinct situations
      1. overlap completely encompasses interval 
           ==> returned interval should be empty
      2. interval completely encompasses overlap
           - if split_interval==True: 
                 return the two sub-intervals on either side of overlap
           - if split_interval==False: 
                 return the first sub-interval which occurs prior to overlap
      3. overlap begins BEFORE interval begins and ends BEFORE interval ends
           ==> return portion of interval occurring after overlap ends
      4. overlap begins AFTER interval begins and ends AFTER interval ends
           ==> return portion of interval occurring before overlap begins
    """
    #--------------------------------------------------
    assert(Utilities.is_object_one_of_types(interval, [list, tuple]))
    assert(Utilities.is_object_one_of_types(overlap, [list, tuple]))
    #-----
    if len(interval)==0:
        return interval
    #-----
    assert(len(interval)==2)
    assert(len(overlap)==2)
    #-----
    assert(interval[1] > interval[0])
    assert(overlap[1]  > overlap[0])
    #-------------------------
    # If the two do not overlap, simply return interval
    if not pd.Interval(interval[0], interval[1]).overlaps(pd.Interval(overlap[0], overlap[1])):
        return interval
    #--------------------------------------------------
    # 4 distinct situations described in function documentation
    #-------------------------
    # 1. overlap completely encompasses interval 
    if (overlap[0] <= interval[0]) and (overlap[1] >= interval[1]):
        return []
    #-------------------------
    # 2. interval completely encompasses overlap
    if (interval[0] <= overlap[0]) and (interval[1] >= overlap[1]):
        intrvl_1 = [interval[0], overlap[0]]
        intrvl_2 = [overlap[1],  interval[1]]
        if split_interval:
            return intrvl_1, intrvl_2
        else:
            return intrvl_1
        
    #-------------------------
    # 3. overlap begins BEFORE interval begins and ends BEFORE interval ends
    #     ==> return portion of interval occurring after overlap ends
    if overlap[0] < interval[0]:
        assert(overlap[1] < interval[1])
        return [overlap[1], interval[1]]
    
    #-------------------------
    # 4. overlap begins AFTER interval begins and ends AFTER interval ends
    #     ==> return portion of interval occurring before overlap begins
    if overlap[0] > interval[0]:
        assert(overlap[1] > interval[1])
        return [interval[0], overlap[0]]
    
    #-------------------------
    # SHOULD NEVER REACH THIS POINT!
    print('In remove_overlap_from_interval: None of 4 distinct situations caught current!\nCRASH IMMINENT!')
    print(f'interval = {interval}')
    print(f'overlap  = {overlap}')
    assert(0)

In [ ]:
def remove_overlap_from_interval(
    interval, 
    overlap, 
    return_removed=False
):
    r"""
    Given an input interval and possible overlap interval, remove overlap from the interval.
    Both interval and overlap should be lists/tuples of length 2 whose elements represent the beginning and
      ending of the interval.
    
    This will NOT split the input interval.
        If the overlap does split the interval, the portion of interval occurring prior to the overlap will be returned.
        The intent of this function is to remove any DOVS events overlapping with the algorithm outage times.
        Therefore, in such a split situation, we would not want anything after the first overlap.
    --------------------------------------------------
    There are basically 4 distinct situations
      1. overlap completely encompasses interval 
           ==> returned interval should be empty
      2. interval completely encompasses overlap
           ==> return the first sub-interval which occurs prior to overlap
      3. overlap begins BEFORE interval begins and ends BEFORE interval ends
           ==> return portion of interval occurring after overlap ends
      4. overlap begins AFTER interval begins and ends AFTER interval ends
           ==> return portion of interval occurring before overlap begins
    """
    #--------------------------------------------------
    assert(Utilities.is_object_one_of_types(interval, [list, tuple]))
    assert(Utilities.is_object_one_of_types(overlap, [list, tuple]))
    #-----
    if len(interval)==0:
        if return_removed:
            return interval, []
        return interval
    #-----
    assert(len(interval)==2)
    assert(len(overlap)==2)
    #-----
    assert(interval[1] > interval[0])
    assert(overlap[1]  > overlap[0])
    #-------------------------
    # If the two do not overlap, simply return interval
    if not pd.Interval(interval[0], interval[1]).overlaps(pd.Interval(overlap[0], overlap[1])):
        rtrn_intrvl = []
        rmvd_intrvl = []
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    #--------------------------------------------------
    # 4 distinct situations described in function documentation
    #-------------------------
    # 1. overlap completely encompasses interval 
    if (overlap[0] <= interval[0]) and (overlap[1] >= interval[1]):
        rtrn_intrvl = []
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    #-------------------------
    # 2. interval completely encompasses overlap
    if (interval[0] <= overlap[0]) and (interval[1] >= overlap[1]):
        rtrn_intrvl = [interval[0], overlap[0]]
        rmvd_intrvl = [overlap[0], interval[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
        
    #-------------------------
    # 3. overlap begins BEFORE interval begins and ends BEFORE interval ends
    #     ==> return portion of interval occurring after overlap ends
    if overlap[0] < interval[0]:
        assert(overlap[1] < interval[1])
        rtrn_intrvl = [overlap[1], interval[1]]
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    
    #-------------------------
    # 4. overlap begins AFTER interval begins and ends AFTER interval ends
    #     ==> return portion of interval occurring before overlap begins
    if overlap[0] > interval[0]:
        assert(overlap[1] > interval[1])
        rtrn_intrvl = [interval[0], overlap[0]]
        rmvd_intrvl = [overlap[0], overlap[1]]
        if return_removed:
            return rtrn_intrvl, rmvd_intrvl
        return rtrn_intrvl
    
    #-------------------------
    # SHOULD NEVER REACH THIS POINT!
    print('In remove_overlap_from_interval: None of 4 distinct situations caught current!\nCRASH IMMINENT!')
    print(f'interval = {interval}')
    print(f'overlap  = {overlap}')
    assert(0)

In [ ]:
def remove_overlaps_from_interval(
    interval, 
    overlaps, 
    return_removed=False
):
    r"""
    Remove multiple possible overlaps from interval
    """
    #-------------------------
    assert(isinstance(overlaps, list))
    assert(Utilities.are_all_list_elements_one_of_types(lst=overlaps, types=[list, tuple]))
    assert(Utilities.are_list_elements_lengths_homogeneous(lst=overlaps, length=2))
    #-------------------------
    return_int = copy.deepcopy(interval)
    removed_intrvls = []
    for overlap_i in overlaps:
        if len(return_int)==0:
            break
        return_int, rmvd_intrvl_i = remove_overlap_from_interval(
            interval       = return_int, 
            overlap        = overlap_i, 
            return_removed = True
        )
        removed_intrvls.append(rmvd_intrvl_i)
    #-------------------------
    if return_removed:
        return return_int, removed_intrvls
    return return_int

In [ ]:
winner_min

In [ ]:
winner_max

In [ ]:
overlap_times

In [ ]:
remove_overlap_from_interval(
    interval = [winner_min, winner_max], 
    overlap = overlap_times[0], 
    return_removed=False
)

In [ ]:
remove_overlap_from_interval(
    interval = [winner_min, winner_max], 
    overlap = overlap_times[0], 
    return_removed=True
)

In [ ]:
r1,r2 = remove_overlaps_from_interval(
    interval = [winner_min, winner_max], 
    overlaps = overlap_times, 
    return_removed=True
)

In [ ]:
r1

In [ ]:
r2

In [ ]:
best_ests_df_w_keep_info = best_ests_df_w_keep_info_OG.copy()

best_ests_df_w_keep_info['winner_min_OG'] = best_ests_df_w_keep_info['winner_min']
best_ests_df_w_keep_info['winner_max_OG'] = best_ests_df_w_keep_info['winner_max']
best_ests_df_w_keep_info['removed_times'] = [[] for _ in range(best_ests_df_w_keep_info.shape[0])]

In [ ]:
best_ests_df_w_keep_info

In [ ]:
best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False]

In [ ]:
t_min_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df_w_keep_info, 
    col = 'winner_min', 
    exact_match=True, 
    assert_single=True
)
t_max_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df_w_keep_info, 
    col = 'winner_max', 
    exact_match=True, 
    assert_single=True
)
rmvd_times_col_idx = Utilities_df.find_idxs_in_highest_order_of_columns(
    df = best_ests_df_w_keep_info, 
    col = 'removed_times', 
    exact_match=True, 
    assert_single=True
)

In [ ]:
# Can't get around doing this row-by-row
for idx_i in range(best_ests_df_w_keep_info.shape[0]):
    t_min_i = best_ests_df_w_keep_info.iloc[idx_i]['winner_min']
    t_max_i = best_ests_df_w_keep_info.iloc[idx_i]['winner_max']
    overlaps_i = best_ests_df_w_keep_info.iloc[idx_i]['overlap_times']
    if overlaps_i is None or len(overlaps_i)==0:
        continue
    #-----
    interval_i, removed_i = remove_overlaps_from_interval(
        interval = [t_min_i, t_max_i], 
        overlaps = overlaps_i, 
        return_removed=True
    )
    #-----
    if len(interval_i)==0:
        best_ests_df_w_keep_info.iloc[idx_i, t_min_col_idx] = None
        best_ests_df_w_keep_info.iloc[idx_i, t_max_col_idx] = None
    else:
        best_ests_df_w_keep_info.iloc[idx_i, t_min_col_idx] = interval_i[0]
        best_ests_df_w_keep_info.iloc[idx_i, t_max_col_idx] = interval_i[1]
    #-----
    # NOTE: Using iat instead of iloc.
    #       Using iloc gives "ValueError: Must have equal len keys and value when setting with an ndarray"
    #         because pandas thinks I am interacting with multiple rows
    #       Using iat resolves the confusion by letting pandas know I am dealing with a single element
    best_ests_df_w_keep_info.iat[idx_i, rmvd_times_col_idx] = removed_i

In [ ]:
best_ests_df_w_keep_info

In [ ]:
best_ests_df_w_keep_info[best_ests_df_w_keep_info['keep']==False]

In [ ]:
fig, ax = Plot_General.default_subplots(fig_num=fig_num)
fig, ax = AMINonVee.plot_usage_around_outage(
    fig=fig, 
    ax=ax, 
    data=ami_df_i, 
    x='starttimeperiod_local', 
    y='value', 
    hue='serialnumber', 
    out_t_beg=dovs_outg_t_beg, 
    out_t_end=dovs_outg_t_end, 
    expand_time=pd.Timedelta('12 hours'), 
    plot_time_beg_end=[dovs_outg_t_beg, dovs_outg_t_end],
    draw_outage_limits_kwargs  = dict(alpha=1.0, linewidth=5.0, ymax=0.1), 
    include_outage_limits_text = dict(
        out_t_beg_text  = 'DOVS 1 Beg.', 
        out_t_beg_ypos  = (0.12, 'ax_coord'), 
        out_t_beg_va    = 'bottom', 
        out_t_beg_ha    = 'center', 
        out_t_beg_color = 'red', 
        #-----
        out_t_end_text  = 'DOVS 1 End', 
        out_t_end_ypos  = (0.12, 'ax_coord'), 
        out_t_end_va    = 'bottom', 
        out_t_end_ha    = 'center', 
        out_t_end_color = 'green', 
    ), 
    only_connect_continuous=True, 
)
ax.axvline(pd.to_datetime('2023-09-04 14:18:00'))